In [ ]:
import numpy as np
from src.wasserstein_barycenters_3d import convolutional_barycenter_3d
from src.utils import point_cloud_to_mesh, plot_mesh

In [ ]:
shape1 = np.load('data/numpy/duck.npy')
shape2 = np.load('data/numpy/torus.npy')
initial_shapes = np.concatenate((shape1[np.newaxis, :, :, :], shape2[np.newaxis, :, :, :]), axis=0)

In [ ]:
N = initial_shapes.shape[1]
t = np.linspace(0, 1, N)
space = np.stack(np.meshgrid(t, t, t), axis=-1).reshape(N**3, -1)

In [ ]:
n_steps = 5
t = np.linspace(0,1,n_steps)
weights = np.array([[s,1-s] for s in t])

In [ ]:
sigma = N//25
tol = 1e-4

In [ ]:
# points clouds to distributions 
for i in range(len(initial_shapes)):
    initial_shapes[i] = initial_shapes[i]/np.sum(initial_shapes[i])

In [ ]:
barycenters = []

for alpha in weights:
    barycenter = convolutional_barycenter_3d(initial_shapes, alpha, sigma=sigma, tol=tol)
    barycenter = barycenter/max(barycenter.flatten())
    barycenters.append(barycenter)

In [ ]:
cloud = barycenters[2] > 0.5
vertices, faces = point_cloud_to_mesh(cloud, space)
plot_mesh(vertices, faces, show_axis=False)